# Making a Linear Regression Model with basic tools 


## Imports

In [1]:
import numpy as np
import torch

## Dataset Making

In [2]:
inputs = np.array([[73,67,43],
                   [91,88,64],
                   [87,134,58],
                   [102,43,37],
                   [69,96,70]],dtype = "float32")
# Making Inputs

In [3]:
targets = np.array([[56,70],
                   [81,101],
                   [119,133],
                   [22,37],
                   [103,119]],dtype = "float32")
# Making Inputs

In [4]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)
#Converting to tensors 

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [5]:
w = torch.randn(2,3,requires_grad=True)
b = torch.randn(2,requires_grad=True)
print(w)
print(b)
# Setting random weights

tensor([[-0.8788, -1.6554,  0.0572],
        [-0.5096,  1.6055,  0.5612]], requires_grad=True)
tensor([-0.3692,  0.8245], requires_grad=True)


## Making a simple linear regression model 

In [6]:
def model(x):
    return x @ w.t() + b
    #making a simple model to generate predictions

In [7]:
preds = model(inputs)
print(preds)

tensor([[-172.9693,   95.3260],
        [-222.3487,  131.6547],
        [-295.3244,  204.1801],
        [-159.0671,   38.6475],
        [-215.9160,  159.0775]], grad_fn=<AddBackward0>)


In [8]:
def predict():
    preds = model(inputs)
    print(preds)

## Making a Mean Square Error or Loss Function

In [9]:
def mse(t1,t2):
    diff = t1-t2
    return torch.sum(diff * diff)/diff.numel()
    #Making it a reusable function 

In [10]:
loss = mse(preds,targets)
print(loss)  #loss before training 

tensor(45886.1406, grad_fn=<DivBackward0>)


## Result Function

In [11]:
preds = model(inputs)
def result():
    print("Predictions")
    print(preds)
    print("Target")
    print(targets)
    print("loss: " , mse(preds,targets)) #For checking the loss
result()
#First Output 

Predictions
tensor([[-172.9693,   95.3260],
        [-222.3487,  131.6547],
        [-295.3244,  204.1801],
        [-159.0671,   38.6475],
        [-215.9160,  159.0775]], grad_fn=<AddBackward0>)
Target
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])
loss:  tensor(45886.1406, grad_fn=<DivBackward0>)


## Computing the gradients

In [12]:
loss.backward()  #Calculating the derivatives of b and w that are in loss 
#the loss is now a quadretic function as there is squaring and etc, going on (in the model)

In [13]:
print(w)
print(w.grad)

tensor([[-0.8788, -1.6554,  0.0572],
        [-0.5096,  1.6055,  0.5612]], requires_grad=True)
tensor([[-24167.9531, -27191.3828, -16462.8828],
        [  2752.8875,   3570.1736,   2009.1492]])


In [14]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)
#setting grads to zero

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


In [15]:
predict()

tensor([[-172.9693,   95.3260],
        [-222.3487,  131.6547],
        [-295.3244,  204.1801],
        [-159.0671,   38.6475],
        [-215.9160,  159.0775]], grad_fn=<AddBackward0>)


In [16]:
loss = mse(preds,targets)
print(loss)  #loss before training 

tensor(45886.1406, grad_fn=<DivBackward0>)


In [17]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-24167.9531, -27191.3828, -16462.8828],
        [  2752.8875,   3570.1736,   2009.1492]])
tensor([-289.3251,   33.7772])


## Adjusting Weights

In [18]:
with torch.no_grad(): #stops tracking the grad as we make the change to it
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [19]:
print(w)
print(b)

tensor([[-0.6371, -1.3835,  0.2218],
        [-0.5371,  1.5698,  0.5411]], requires_grad=True)
tensor([-0.3663,  0.8242], requires_grad=True)


In [20]:
loss = mse(preds,targets)
print(loss)  #loss after training for once

tensor(45886.1406, grad_fn=<DivBackward0>)


## Training the Model

In [21]:
#Train for 1000 Epochs
for i in range(1000):
    preds = model(inputs)
    loss = mse(preds,targets)
    loss.backward()
    with torch.no_grad(): #stops tracking the grad as we make the change to it
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

## Result

In [22]:
print("loss: " , mse(preds,targets)) #loss after training for 1000 Epoch 

loss:  tensor(4.0899, grad_fn=<DivBackward0>)


In [23]:
result()

Predictions
tensor([[ 56.9865,  70.6170],
        [ 83.4022,  99.0778],
        [116.2607, 136.0875],
        [ 20.5276,  37.8199],
        [104.2516, 116.0611]], grad_fn=<AddBackward0>)
Target
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])
loss:  tensor(4.0899, grad_fn=<DivBackward0>)


-

# Making the same Model using tools from Torch

-

In [24]:
import torch.nn as nn

In [25]:
inputs = np.array([
    [87, 134, 58],[102, 43, 37],[91, 88, 64],[69, 96, 70],[73, 67, 43],
    [73, 67, 43],[91, 88, 64],[69, 96, 70],[87, 134, 58],[102, 43, 37],
    [102, 43, 37],[87, 134, 58],[91, 88, 64],[69, 96, 70],[73, 67, 43],
    [69, 96, 70],[91, 88, 64],[102, 43, 37],[87, 134, 58],[73, 67, 43],
    [91, 88, 64],[102, 43, 37],[73, 67, 43],[69, 96, 70],[87, 134, 58]]
                  ,dtype="float32")

targets = np.array([
    [119, 133],[22, 37],[81, 101],[103, 119],[56, 70],    
    [56, 70],[81, 101],[103, 119],[119, 133],[22, 37],    
    [22, 37],[119, 133],[81, 101],[103, 119],[56, 70],    
    [103, 119],[81, 101],[22, 37],[119, 133],[56, 70],    
    [81, 101],[22, 37],[56, 70],[103, 119],[119, 133]]
                   , dtype="float32")

In [26]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [27]:
from torch.utils.data import TensorDataset

In [28]:
train_ds = TensorDataset(inputs,targets)
train_ds[0:3]

(tensor([[ 87., 134.,  58.],
         [102.,  43.,  37.],
         [ 91.,  88.,  64.]]),
 tensor([[119., 133.],
         [ 22.,  37.],
         [ 81., 101.]]))

In [29]:
from torch.utils.data import DataLoader

In [30]:
batch_size = 5
train_dl = DataLoader(train_ds,batch_size,shuffle=True)

In [31]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 87., 134.,  58.],
        [ 73.,  67.,  43.],
        [102.,  43.,  37.],
        [ 91.,  88.,  64.],
        [ 91.,  88.,  64.]])
tensor([[119., 133.],
        [ 56.,  70.],
        [ 22.,  37.],
        [ 81., 101.],
        [ 81., 101.]])


In [32]:
#Define Model
model = nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.0343, -0.3308, -0.0394],
        [-0.3331, -0.2036,  0.3865]], requires_grad=True)
Parameter containing:
tensor([-0.5757,  0.0488], requires_grad=True)


In [33]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.0343, -0.3308, -0.0394],
         [-0.3331, -0.2036,  0.3865]], requires_grad=True),
 Parameter containing:
 tensor([-0.5757,  0.0488], requires_grad=True)]

In [34]:
#Generate Predictions
preds = model(inputs)
preds

tensor([[-44.2144, -33.7955],
        [-12.7649, -28.3775],
        [-29.0948, -23.4414],
        [-32.7323, -15.4241],
        [-21.9359, -21.2871],
        [-21.9359, -21.2871],
        [-29.0948, -23.4414],
        [-32.7323, -15.4241],
        [-44.2144, -33.7955],
        [-12.7649, -28.3775],
        [-12.7649, -28.3775],
        [-44.2144, -33.7955],
        [-29.0948, -23.4414],
        [-32.7323, -15.4241],
        [-21.9359, -21.2871],
        [-32.7323, -15.4241],
        [-29.0948, -23.4414],
        [-12.7649, -28.3775],
        [-44.2144, -33.7955],
        [-21.9359, -21.2871],
        [-29.0948, -23.4414],
        [-12.7649, -28.3775],
        [-21.9359, -21.2871],
        [-32.7323, -15.4241],
        [-44.2144, -33.7955]], grad_fn=<AddmmBackward0>)

## Defining Loss Function

In [35]:
import torch.nn.functional as F

In [36]:
loss_fn = F.mse_loss

In [37]:
loss = loss_fn(model(inputs),targets)
print(loss)

tensor(13844.9434, grad_fn=<MseLossBackward0>)


## Define Optimizer

In [38]:
#SGD Stocastic Gradient Descnet 
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [39]:
#Utility function to train the model
def fit(num_epochs, model,loss_fn,opt,train_dl):
    #Repeat for given number of epochs
    for epoch in range(num_epochs):
        #Train with batches of data
        for xb, yb in train_dl:
            #Generate Predicitons
            pred = model(xb)
            #Calculate loss
            loss = loss_fn(pred,yb)
            #Compute Gradient
            loss.backward()
            #Update Parameters using gradients
            opt.step()
            #Reset the gradients
            opt.zero_grad()
        #print the progress
        if (epoch + 1) % 10 == 0:
            print('Epoch [{}/{}], Loss : {:.4f}'.format(epoch+1,num_epochs,loss))

## Training The Model

In [44]:
fit(100,model,loss_fn,opt,train_dl)

Epoch [10/100], Loss : 3.0441
Epoch [20/100], Loss : 1.3753
Epoch [30/100], Loss : 0.8625
Epoch [40/100], Loss : 1.1602
Epoch [50/100], Loss : 1.2218
Epoch [60/100], Loss : 1.2597
Epoch [70/100], Loss : 0.9645
Epoch [80/100], Loss : 1.0597
Epoch [90/100], Loss : 1.0145
Epoch [100/100], Loss : 0.3672


## Result

In [41]:
def result_(p,t):
    print("Predictions")
    print(p)
    print("Target")
    print(t)
    print("loss: " , mse(p,t))

In [45]:
pred = model(inputs)
result_(pred,targets)

Predictions
tensor([[119.8627, 133.3568],
        [ 21.4244,  37.1318],
        [ 81.7054, 100.4782],
        [100.7359, 118.7447],
        [ 57.1335,  70.3438],
        [ 57.1335,  70.3438],
        [ 81.7054, 100.4782],
        [100.7359, 118.7447],
        [119.8627, 133.3568],
        [ 21.4244,  37.1318],
        [ 21.4244,  37.1318],
        [119.8627, 133.3568],
        [ 81.7054, 100.4782],
        [100.7359, 118.7447],
        [ 57.1335,  70.3438],
        [100.7359, 118.7447],
        [ 81.7054, 100.4782],
        [ 21.4244,  37.1318],
        [119.8627, 133.3568],
        [ 57.1335,  70.3438],
        [ 81.7054, 100.4782],
        [ 21.4244,  37.1318],
        [ 57.1335,  70.3438],
        [100.7359, 118.7447],
        [119.8627, 133.3568]], grad_fn=<AddmmBackward0>)
Target
tensor([[119., 133.],
        [ 22.,  37.],
        [ 81., 101.],
        [103., 119.],
        [ 56.,  70.],
        [ 56.,  70.],
        [ 81., 101.],
        [103., 119.],
        [119., 133.],
      